<a href="https://colab.research.google.com/github/akingulpinar/CNG562_MalariaCellImages/blob/master/Gulpinar_Malaria_Cell_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Malaria Cell Images



In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle 

In [0]:
import json
token = {"username":"akingulpinar","key":"766a7554ab5f8eed8f08cdd12ea6e636"}
!mkdir -p .kaggle/
file = open('/content/.kaggle/kaggle.json', 'w+')
json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria
!rm -rf cell_images/

In [0]:
!unzip cell-images-for-detecting-malaria.zip

In [0]:
import shutil
import os
shutil.move(os.getcwd()+"/cell_images/Parasitized",os.getcwd()+"/cell_images/training_set/Parasitized")
shutil.move(os.getcwd()+"/cell_images/Uninfected",os.getcwd()+"/cell_images/training_set/Uninfected")

In [0]:
!mkdir -p /content/cell_images/test_set/Parasitized
!mkdir -p /content/cell_images/test_set/Uninfected

In [0]:
def split_train_test_images(percentage):
    import random
    import shutil
    parasitizedTrainFolder =os.getcwd()+'/cell_images/training_set/Parasitized/'
    uninftectedTrainFolder = os.getcwd()+'/cell_images/training_set/Uninfected/'
    parasitizedTestFolder =os.getcwd()+'/cell_images/test_set/Parasitized/'
    uninftectedTestFolder =os.getcwd()+'/cell_images/test_set/Uninfected/'
    
    parasitized = os.listdir(parasitizedTrainFolder)
    uninfected = os.listdir(uninftectedTrainFolder)

    for i  in  range(int((np.size(parasitized) * percentage))):
        x = random.choice(os.listdir(parasitizedTrainFolder))
        shutil.move(parasitizedTrainFolder + x,parasitizedTestFolder)
        
    for i  in  range(int((np.size(uninfected) * percentage))):
        x = random.choice(os.listdir(uninftectedTrainFolder))
        shutil.move(uninftectedTrainFolder + x,uninftectedTestFolder)

In [0]:
import numpy as np
import pandas as pd
split_train_test_images(0.2)

In [0]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
pprint.pprint(devices)

In [0]:
#Convolutional Neural Network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

#Intialization
classifier = Sequential()

#Convolution
classifier.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))

#Max Pooling / Downsampling
classifier.add(MaxPooling2D(pool_size=(2,2)))

#2nd Convolution 
classifier.add(Conv2D(128,(3,3),activation='relu'))

#2nd Max Pooling / Downsampling
classifier.add(MaxPooling2D(pool_size=(2,2)))

#Dropout
classifier.add(Dropout(rate=0.25))

#Flatten
classifier.add(Flatten())

#Full Connection
classifier.add(Dense(128, activation='relu'))
classifier.add(Dropout(rate=0.5))
classifier.add(Dense(1, activation='sigmoid'))

In [0]:
classifier.summary()

In [0]:
opt = tf.train.AdamOptimizer(0.001)
classifier.compile(opt,loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('cell_images/training_set', target_size=(64, 64), batch_size=32, class_mode='binary')
test_set = test_datagen.flow_from_directory('cell_images/test_set', target_size=(64, 64), batch_size=32, class_mode='binary')

In [0]:
TPU_ADDRESS = "grpc://" + os.environ['COLAB_TPU_ADDR']
resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
dist_strategy = tf.contrib.tpu.TPUDistributionStrategy(resolver)
tpu_model = tf.contrib.tpu.keras_to_tpu_model(classifier,strategy=dist_strategy)

In [0]:
tpu_model.fit_generator(training_set, steps_per_epoch=22046, epochs=10, validation_data=test_set, validation_steps=5512)